In [18]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest

# Загрузка данных
file_path = r"C:\Users\SWS-M\Downloads\marketing_AB\marketing_AB.csv" #Датасет большой для чтения с гита, пришлось с компухтера читать
data = pd.read_csv(file_path)

# Очистка данных: удаляем ненужные колонки
data_cleaned = data.drop(columns=['Unnamed: 0'], errors='ignore')

In [19]:
# Преобразуем категориальные переменные в дамми-переменные
categorical_cols = ['test group', 'most ads day']
data_cleaned = pd.get_dummies(data_cleaned, columns=categorical_cols, drop_first=True)

# Преобразуем все булевы переменные в числовые
for col in data_cleaned.select_dtypes(include='bool').columns:
    data_cleaned[col] = data_cleaned[col].astype(int)

# Проверим на пропущенные значения и заполним их средними значениями
if data_cleaned.isnull().any().any():
    data_cleaned = data_cleaned.fillna(data_cleaned.mean())

# Подготовка данных для логистической регрессии
X = data_cleaned.drop(columns=['converted', 'user id'])
y = data_cleaned['converted'].astype(int)

# Проверка типов данных
# print("Типы данных X:")
# print(X.dtypes)
# print("\nУникальные значения y:")
# print(y.unique())

# Логистическая регрессия
logit_model = sm.Logit(y, sm.add_constant(X)).fit()
print("\nLogistic regression summary:")
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.108329
         Iterations 8

Logistic regression summary:
                           Logit Regression Results                           
Dep. Variable:              converted   No. Observations:               588101
Model:                          Logit   Df Residuals:                   588091
Method:                           MLE   Df Model:                            9
Date:                Mon, 25 Nov 2024   Pseudo R-squ.:                 0.08025
Time:                        21:32:37   Log-Likelihood:                -63708.
converged:                       True   LL-Null:                       -69267.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -4.7613      0.038   -126.62

### 1. Какой тип рекламы был успешнее?
В модели есть переменная test group_psa, которая указывает, принадлежит ли пользователь к группе с социальной рекламой (PSA) или нет.

Коэффициент для test group_psa: -0.3779

Этот коэффициент отрицательный, что указывает на то, что пользователи, видевшие социальную рекламу (PSA), с меньшей вероятностью конвертировались по сравнению с пользователями в контрольной группе.

P-значение: 0.000

Значение меньше стандартного уровня значимости (0.05), что подтверждает статистическую значимость результата.
Вывод: Традиционная реклама была успешнее социальной рекламы.

### 2. Дополнительные переменные и гетерогенные эффекты
Включение других переменных (например, время показа рекламы, день недели) позволяет понять, как различные факторы влияют на вероятность конверсии.

Ключевые результаты:

total ads: 0.0103 (p-value < 0.05):

Чем больше рекламы видит пользователь, тем выше вероятность конверсии. Это логично, поскольку увеличение экспозиции приводит к росту вовлеченности.

most ads hour: 0.0337 (p-value < 0.05):

Часы с максимальным числом показов положительно влияют на вероятность конверсии. Вероятно, определенные часы суток более удачны для показа рекламы.
День недели:

Monday: Положительный эффект (0.4725), статистически значимый (p-value < 0.05).

Tuesday: Сильный положительный эффект (0.4438), значимый.

Sunday: Умеренный положительный эффект (0.2174), значимый.

Thursday: Положительный эффект (0.0679), на грани значимости (p-value = 0.046).

Saturday: Эффект незначимый (p-value = 0.522).

Вывод: Дни недели оказывают влияние на конверсию. Наиболее успешные дни — понедельник и вторник.

### 3. Выводы об эффективности с подтверждением статистическими тестами

- Проверка гипотезы о влиянии типа рекламы:

Для переменной test group_psa p-value < 0.05. Мы отвергаем нулевую гипотезу о равной эффективности социальной и традиционной рекламы. Традиционная реклама эффективнее.

- Проверка влияния дня недели:

Коэффициенты для большинства дней недели (кроме субботы) статистически значимы, что подтверждает их влияние на конверсии. Это стоит учитывать при планировании показа рекламы.